# VTK - ITK Project : Etude longitudinale de l'évolution d'une tumeur

### Par Raphael Duhen, Mael Conan et Andrews Nigel

In [1]:
# Imports
import vtk
import itk
import matplotlib.pyplot as plt
import numpy as np

In [2]:
reader = vtk.vtkNrrdReader()
reader.SetFileName("Data/case6_gre1.nrrd")

window = vtk.vtkRenderWindow()
renderer = vtk.vtkRenderer()

window.AddRenderer(renderer)

interactor = vtk.vtkRenderWindowInteractor()
window.SetInteractor(interactor)

contour = vtk.vtkContourFilter()
contour.SetInputConnection(reader.GetOutputPort())
contour.SetValue(0, 135)

contourMapper = vtk.vtkPolyDataMapper()
contourMapper.SetInputConnection(contour.GetOutputPort())
contourMapper.ScalarVisibilityOff()

contourActor = vtk.vtkActor()
contourActor.SetMapper(contourMapper)

renderer.AddActor(contourActor)

window.Render()
interactor.Start()

In [3]:
# Define types
PixelType = itk.ctype('float')
Dimension = 3
ImageType = itk.Image[PixelType, Dimension]
TransformType = itk.Euler3DTransform[itk.D]

In [4]:
# Load images
image = itk.imread("Data/case6_gre1.nrrd", pixel_type=PixelType)
image2 = itk.imread("Data/case6_gre2.nrrd", pixel_type=PixelType) 

In [5]:
initial_transform = TransformType.New()

# Set initial transform parameters
initial_transform.SetRotation(0.0, 0.0, 0.0)
initial_transform.SetTranslation(0.0)

# Set optimizer parameters
optimizer = itk.RegularStepGradientDescentOptimizerv4.New(
    LearningRate=4,
    MinimumStepLength=0.001,
    RelaxationFactor=0.5,
    NumberOfIterations=200,
)

# Set metric parameters
metric = itk.MeanSquaresImageToImageMetricv4[ImageType, ImageType].New()

# Set interpolator parameters
interpolator = itk.LinearInterpolateImageFunction[ImageType, itk.D].New()

# Set registration parameters
registration = itk.ImageRegistrationMethodv4[ImageType, ImageType].New(
    Metric=metric,
    Optimizer=optimizer,
    InitialTransform=initial_transform,
    FixedImage=image,
    MovingImage=image2,
)

# Start registration
registration.Update()

In [6]:

# Transform image
transformed_image = itk.resample_image_filter(
    image2,
    transform=registration.GetTransform(),
    use_reference_image=True,
    reference_image=image,
    default_pixel_value=100,
)

In [8]:

# Plot images
# plt.figure(figsize=(10, 10))
# 
# plt.subplot(1, 2, 1)
# plt.imshow(itk.array_from_image(image))
# plt.title("Fixed image")
# 
# plt.subplot(1, 2, 2)
# plt.imshow(itk.array_from_image(transformed_image))
# plt.title("Moving image transformed")
# 
# plt.show()

In [9]:
# Save image
itk.imwrite(transformed_image, "Data/transformed_image.nrrd")

In [12]:
# Get final parameters
final_parameters = registration.GetOutput().Get().GetParameters()

# # Get final metric value
# final_metric_value = registration.GetMetricValue()

# Get final number of iterations
final_number_of_iterations = optimizer.GetCurrentIteration()

# Print results
print("Result = ", final_parameters)
# print("Metric value = ", final_metric_value)
print("Number of iterations = ", final_number_of_iterations)

Result =  <itk.itkOptimizerParametersPython.itkOptimizerParametersD; proxy of <Swig Object of type 'itkOptimizerParametersD *' at 0x15e46c360> >
Number of iterations =  200
